## 1. Environment Setup

In [97]:
# Environment setup
from pathlib import Path
from typing import Optional

try:
    from dotenv import load_dotenv
    DOTENV_AVAILABLE = True
except Exception:
    DOTENV_AVAILABLE = False

# Helper to find project root
def _find_root(start: Optional[Path] = None) -> Path:
    p = start or Path.cwd()
    for _ in range(6):
        if (p / 'data').exists() or (p / '.git').exists() or (p / 'notebooks').exists():
            return p
        p = p.parent
    return Path.cwd()

# Resolve project directories consistently
ROOT = _find_root()
DATA_DIR = ROOT / 'data' / 'raw'
INTERIM_DIR = ROOT / 'data' / 'interim'
PROCESSED_DIR = ROOT / 'data' / 'processed'
FIG_DIR = ROOT / 'reports' / 'figures'
for d in [DATA_DIR, INTERIM_DIR, PROCESSED_DIR, FIG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"\n🎯 Environment setup complete")
print(f"   ROOT: {ROOT}")
print(f"   DATA_DIR: {DATA_DIR}")


🎯 Environment setup complete
   ROOT: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks
   DATA_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\raw


## 2. Helper Functions

In [98]:
# Helper functions for scraping
from typing import Optional
import random
import time
from pathlib import Path
import requests

_USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0 Safari/537.36",
]

def find_repo_root(start: Optional[Path] = None) -> Path:
    p = start or Path.cwd()
    for _ in range(6):
        if (p / 'data').exists() or (p / '.git').exists() or (p / 'notebooks').exists():
            return p
        p = p.parent
    return Path.cwd()

def ensure_environment():
    global ROOT, DATA_DIR, INTERIM_DIR, PROCESSED_DIR, FIG_DIR
    if 'ROOT' not in globals() or not isinstance(ROOT, Path) or not (ROOT / 'data').exists():
        root_guess = find_repo_root(Path.cwd())
        if not (root_guess / 'data').exists() and (root_guess.parent / 'data').exists():
            root_guess = root_guess.parent
        ROOT = root_guess
    DATA_DIR = ROOT / 'data' / 'raw'
    INTERIM_DIR = ROOT / 'data' / 'interim'
    PROCESSED_DIR = ROOT / 'data' / 'processed'
    FIG_DIR = ROOT / 'reports' / 'figures'
    for d in [DATA_DIR, INTERIM_DIR, PROCESSED_DIR, FIG_DIR]:
        d.mkdir(parents=True, exist_ok=True)
    return ROOT, DATA_DIR, INTERIM_DIR, PROCESSED_DIR, FIG_DIR


def http_get(url: str, headers: Optional[dict] = None, retries: int = 3, timeout: int = 30) -> str:
    last_err = None
    sess = requests.Session()
    for attempt in range(1, retries + 1):
        ua = random.choice(_USER_AGENTS)
        hdrs = {"User-Agent": ua, "Accept-Language": "en-US,en;q=0.9"}
        if headers:
            hdrs.update(headers)
        try:
            resp = sess.get(url, headers=hdrs, timeout=timeout)
            resp.raise_for_status()
            return resp.text
        except Exception as e:
            last_err = e
            time.sleep(0.8 * attempt)
    raise last_err  # type: ignore


def save_csv(df: 'pd.DataFrame', path: Path, **to_csv_kwargs):
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False, encoding=to_csv_kwargs.get('encoding', 'utf-8-sig'))
    print(f"Saved: {path}")

print("✅ Helper functions loaded")

✅ Helper functions loaded


## 3. Wikipedia Playoffs Scraper Functions

Scrapes playoff results from Wikipedia. The Israeli Premier League splits into two playoff groups:
- **Championship round**: Top 6 teams compete for title
- **Relegation round**: Bottom teams fight to avoid relegation

Both use results matrix format similar to regular season.

In [99]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Reusable regex to find a score anywhere in cell text (robust to footnotes)
_SCORE_RE = re.compile(r"(\d+)\s*[–-]\s*(\d+)")

def _find_round_header(soup: BeautifulSoup, round_type: str):
    target_id = "Championship_round_results" if round_type == "championship" else "Relegation_round_results"
    node = soup.find(id=target_id)
    if node:
        heading = node if node.name in ("h2","h3","h4") else node.find_parent(["h2","h3","h4"])
        if heading:
            return heading
    wanted_text = ("Championship round results" if round_type=="championship" else "Relegation round results").lower()
    for hdr in soup.find_all(["h2","h3","h4"]):
        if hdr.get_text(" ", strip=True).lower() == wanted_text:
            return hdr
    section_names = ["Championship round", "Top playoff"] if round_type == "championship" else ["Relegation round", "Bottom playoff"]
    section_hdr = None
    lower_section_names = [s.lower() for s in section_names]
    for hdr in soup.find_all(["h2","h3"]):
        txt = hdr.get_text(" ", strip=True).strip().lower()
        if txt in lower_section_names:
            section_hdr = hdr
            break
    if section_hdr:
        current = section_hdr
        while True:
            current = current.find_next_sibling()
            if current is None:
                break
            if current.name in ("h2","h3","h4"):
                text = current.get_text(" ", strip=True).strip().lower()
                if text == "results":
                    return current
                if text in ("table", "standings") or text in lower_section_names:
                    continue
                break
            sub = current.find(["h3","h4"], string=lambda s: isinstance(s, str) and s.strip().lower()=="results")
            if sub:
                return sub
        return section_hdr
    for hdr in soup.find_all(["h2","h3","h4"]):
        text = hdr.get_text(" ", strip=True).lower()
        if round_type == "championship":
            if ("championship" in text or re.search(r"top\s*play[- ]?off", text)) and "results" in text:
                return hdr
        else:
            if ("relegation" in text or re.search(r"bottom\s*play[- ]?off", text)) and "results" in text:
                return hdr
    return None

def _find_matrix_after(header_node):
    start = header_node
    parent = getattr(header_node, 'parent', None)
    parent_classes = set((parent.get('class') or [])) if parent else set()
    if parent and ('mw-heading' in parent_classes or any(c.startswith('mw-heading') for c in parent_classes)):
        start = parent
    current = start
    while True:
        current = current.find_next_sibling()
        if current is None:
            break
        if current.name in ("h2","h3","h4"):
            break
        if 'mw-heading' in set(current.get('class') or []) or any(c.startswith('mw-heading') for c in (current.get('class') or [])):
            break
        if current.name == "table" and "wikitable" in (current.get("class") or []):
            first_row = current.find("tr")
            first_cell = first_row.find("th") if first_row else None
            header_text = first_cell.get_text(" ", strip=True) if first_cell else ""
            if re.search(r"Home\s*[\\/]\s*Away|Home.*Away", header_text, re.IGNORECASE):
                return current
        nested = current.find("table", class_=re.compile(r"\bwikitable\b"))
        while nested:
            first_row = nested.find("tr")
            first_cell = first_row.find("th") if first_row else None
            header_text = first_cell.get_text(" ", strip=True) if first_cell else ""
            if re.search(r"Home\s*[\\/]\s*Away|Home.*Away", header_text, re.IGNORECASE):
                return nested
            nested = nested.find_next("table", class_=re.compile(r"\bwikitable\b"))
    return None

def _find_round_table(soup: BeautifulSoup, round_type: str):
    titles = ["Championship round table", "Top playoff table"] if round_type=="championship" else ["Relegation round table", "Bottom playoff table"]
    for hdr in soup.find_all(["h2","h3","h4"]):
        txt = hdr.get_text(" ", strip=True).strip().lower()
        if txt in [t.lower() for t in titles]:
            start = hdr.parent if hdr.parent and 'mw-heading' in set(hdr.parent.get('class') or []) else hdr
            node = start.find_next_sibling()
            tbl = None
            if node and node.name == 'table' and 'wikitable' in (node.get('class') or []):
                tbl = node
            else:
                tbl = start.find_next("table", class_=re.compile(r"\bwikitable\b"))
            return tbl
    return None

def _find_results_by_round_table(soup: BeautifulSoup, round_type: str):
    section_names = ["Championship round", "Top playoff"] if round_type=="championship" else ["Relegation round", "Bottom playoff"]
    lower_section_names = [s.lower() for s in section_names]
    section_hdr = None
    for hdr in soup.find_all(["h2","h3"]):
        txt = hdr.get_text(" ", strip=True).strip().lower()
        if txt in lower_section_names:
            section_hdr = hdr
            break
    if not section_hdr:
        return None
    current = section_hdr
    results_hdr = None
    while True:
        current = current.find_next_sibling()
        if current is None:
            break
        if current.name in ("h2","h3","h4"):
            text = current.get_text(" ", strip=True).strip().lower()
            if text == "results":
                results_hdr = current
                break
            if text in ("table", "standings") or text in lower_section_names:
                continue
            break
        sub = current.find(["h3","h4"], string=lambda s: isinstance(s, str) and s.strip().lower()=="results")
        if sub:
            results_hdr = sub
            break
    if not results_hdr:
        return None
    return _find_matrix_after(results_hdr)

def _extract_score(text: str):
    m = _SCORE_RE.search(text)
    if not m:
        return None
    return int(m.group(1)), int(m.group(2))

def _detect_home_team_from_cells(cells, start_idx_row: int):
    # Prefer a linked team cell within the left-side columns
    for j in range(0, max(start_idx_row, 0)):
        a = cells[j].find('a') if hasattr(cells[j], 'find') else None
        if a:
            t = a.get_text(" ", strip=True)
            if t:
                return TEAM_NAME_MAP.get(t, t)
        txt = cells[j].get_text(" ", strip=True)
        if txt and not txt.isdigit():
            return TEAM_NAME_MAP.get(txt, txt)
    # Fallbacks
    if len(cells) > 1:
        return TEAM_NAME_MAP.get(cells[1].get_text(" ", strip=True), cells[1].get_text(" ", strip=True))
    return TEAM_NAME_MAP.get(cells[0].get_text(" ", strip=True), cells[0].get_text(" ", strip=True))

def _parse_embedded_matrix_table(tbl: 'BeautifulSoup', season_str: str, season_year: int, round_type: str):
    rows = tbl.find_all("tr")
    if not rows:
        return None
    header_row = None
    for tr in rows[:6]:
        ths = tr.find_all("th")
        tokens = [th.get_text(" ", strip=True) for th in ths]
        short = [t for t in tokens if 1 <= len(t) <= 4]
        if len(short) >= 4:
            header_row = tr
            break
    if header_row is None:
        return None
    headers = [th.get_text(" ", strip=True) for th in header_row.find_all("th")]
    # Count how many grid columns (abbreviations) appear in the header row
    left_cols = {"Pos", "Team", "Pld", "W", "D", "L", "GF", "GA", "GD", "Pts", "Qualification"}
    start_idx_guess = 0
    for i, h in enumerate(headers):
        if h in left_cols:
            start_idx_guess = i + 1
        else:
            start_idx_guess = i
            break
    away_abbr = headers[start_idx_guess:]
    away_len = len(away_abbr)
    if away_len < 3:
        return None
    # Normalize away names now
    away_names = [TEAM_NAME_MAP.get(h, h) for h in away_abbr]

    matches = []
    start_collect = False
    for tr in rows:
        if tr == header_row:
            start_collect = True
            continue
        if not start_collect:
            continue
        tds = tr.find_all(["th","td"])
        if len(tds) < away_len + 2:  # need at least team + grid
            continue
        # Align grid to end of row: grid occupies the last `away_len` cells
        start_idx_row = len(tds) - away_len
        home_team = _detect_home_team_from_cells(tds, start_idx_row)
        for k in range(away_len):
            ab = away_names[k]
            cell = tds[start_idx_row + k]
            score_text = cell.get_text(" ", strip=True)
            pair = _extract_score(score_text)
            if pair:
                hg, ag = pair
                matches.append({
                    "season": season_str,
                    "season_year": season_year,
                    "playoff_type": round_type,
                    "home_team": home_team,
                    "away_team": ab,
                    "home_goals": int(hg),
                    "away_goals": int(ag)
                })
    if not matches:
        return None
    df = pd.DataFrame(matches)
    df['goal_diff'] = df['home_goals'] - df['away_goals']
    df['result'] = df['goal_diff'].apply(lambda x: 'H' if x > 0 else ('A' if x < 0 else 'D'))
    df['home_points'] = df['result'].map({'H': 3, 'D': 1, 'A': 0}).astype(int)
    df['away_points'] = df['result'].map({'A': 3, 'D': 1, 'H': 0}).astype(int)
    return df[['season','season_year','playoff_type','home_team','away_team','home_goals','away_goals','goal_diff','result','home_points','away_points']]

def _special_old_format_matrix(soup: BeautifulSoup, round_type: str):
    target_section = 'Top playoff' if round_type=='championship' else 'Bottom playoff'
    sec = None
    for h in soup.find_all(['h3','h4']):
        if h.get_text(' ', strip=True).strip().lower() == target_section.lower():
            sec = h
            break
    if not sec:
        return None
    res = sec.find_next(lambda t: t.name in ('h3','h4') and t.get_text(' ', strip=True).strip().lower()=='results')
    if not res:
        return None
    return _find_matrix_after(res)

def _parse_matrix_table(table, season_str, season_year, round_type):
    rows = table.find_all("tr")
    if not rows:
        return None
    raw_headers = [th.get_text(strip=True) for th in rows[0].find_all("th")][1:] if rows else []
    team_names = [TEAM_NAME_MAP.get(h, h) for h in raw_headers]
    matches = []
    for row in rows[1:]:
        cells = row.find_all(["th", "td"])
        if len(cells) < len(team_names) + 1:
            continue
        home_team = TEAM_NAME_MAP.get(cells[0].get_text(strip=True), cells[0].get_text(strip=True))
        for idx, cell in enumerate(cells[1:]):
            if idx >= len(team_names):
                break
            away_team = team_names[idx]
            score_text = cell.get_text(" ", strip=True)
            pair = _extract_score(score_text)
            if pair:
                hg, ag = pair
                matches.append({
                    "season": season_str,
                    "season_year": season_year,
                    "playoff_type": round_type,
                    "home_team": home_team,
                    "away_team": away_team,
                    "home_goals": int(hg),
                    "away_goals": int(ag)
                })
    if not matches:
        return None
    df = pd.DataFrame(matches)
    df['goal_diff'] = df['home_goals'] - df['away_goals']
    df['result'] = df['goal_diff'].apply(lambda x: 'H' if x > 0 else ('A' if x < 0 else 'D'))
    df['home_points'] = df['result'].map({'H': 3, 'D': 1, 'A': 0}).astype(int)
    df['away_points'] = df['result'].map({'A': 3, 'D': 1, 'H': 0}).astype(int)
    return df[['season','season_year','playoff_type','home_team','away_team','home_goals','away_goals','goal_diff','result','home_points','away_points']]

def scrape_playoff_round(season_year: int, round_type: str):
    season_str = f"{season_year}/{str(season_year+1)[-2:]}"
    url = f"https://en.wikipedia.org/wiki/{season_year}%E2%80%93{str(season_year+1)[-2:]}_Israeli_Premier_League"
    round_name = "Championship" if round_type == "championship" else "Relegation"
    print(f"Fetching {season_str} {round_name} round... ", end="", flush=True)
    try:
        html = http_get(url)
        if not html:
            print("❌ (empty HTML)")
            return None
        soup = BeautifulSoup(html, "html.parser")
        df = None
        if season_year <= 2014:
            tbl = _special_old_format_matrix(soup, round_type)
            if tbl:
                df = _parse_matrix_table(tbl, season_str, season_year, round_type)
        if df is None:
            header = _find_round_header(soup, round_type)
            if header:
                table = _find_matrix_after(header)
                if table:
                    df = _parse_matrix_table(table, season_str, season_year, round_type)
        if df is None:
            rbr = _find_results_by_round_table(soup, round_type)
            if rbr:
                df = _parse_matrix_table(rbr, season_str, season_year, round_type)
        if df is None:
            tbl = _find_round_table(soup, round_type)
            if tbl:
                df = _parse_embedded_matrix_table(tbl, season_str, season_year, round_type)
        if df is None:
            print("❌ (no results found)")
            return None
        print(f"✓ ({len(df)} matches)")
        return df
    except Exception as e:
        print(f"❌ ({str(e)[:60]}...")
        return None

def scrape_season_playoffs(season_year: int):
    championship_df = scrape_playoff_round(season_year, 'championship')
    time.sleep(0.3)
    relegation_df = scrape_playoff_round(season_year, 'relegation')
    return championship_df, relegation_df

print("✅ Playoff scraper functions ready (embedded grid alignment fixed)")

✅ Playoff scraper functions ready (embedded grid alignment fixed)


In [100]:
# Validation for recent seasons (2020/21–2024/25)
recent_years = [2020, 2021, 2022, 2023, 2024]
rows = []
for y in recent_years:
    champ, releg = scrape_season_playoffs(y)
    rows.append({
        'season': f"{y}/{str(y+1)[-2:]}",
        'championship_matches': (len(champ) if champ is not None else None),
        'relegation_matches': (len(releg) if releg is not None else None)
    })
import pandas as _pd
print("\nRecent Seasons Validation:")
_pd.DataFrame(rows)

Fetching 2020/21 Championship round... ✓ (30 matches)
✓ (30 matches)
Fetching 2020/21 Relegation round... Fetching 2020/21 Relegation round... ✓ (28 matches)
Fetching 2021/22 Championship round... ✓ (28 matches)
Fetching 2021/22 Championship round... ✓ (30 matches)
✓ (30 matches)
Fetching 2021/22 Relegation round... Fetching 2021/22 Relegation round... ✓ (28 matches)
Fetching 2022/23 Championship round... ✓ (28 matches)
Fetching 2022/23 Championship round... ✓ (30 matches)
✓ (30 matches)
Fetching 2022/23 Relegation round... Fetching 2022/23 Relegation round... ✓ (28 matches)
Fetching 2023/24 Championship round... ✓ (28 matches)
Fetching 2023/24 Championship round... ✓ (30 matches)
✓ (30 matches)
Fetching 2023/24 Relegation round... Fetching 2023/24 Relegation round... ✓ (28 matches)
Fetching 2024/25 Championship round... ✓ (28 matches)
Fetching 2024/25 Championship round... ✓ (30 matches)
✓ (30 matches)
Fetching 2024/25 Relegation round... Fetching 2024/25 Relegation round... ✓ (28 mat

,season,championship_matches,relegation_matches
0,2020/21,30,28
1,2021/22,30,28
2,2022/23,30,28
3,2023/24,30,28
4,2024/25,30,28


## 4. Multi-Season Playoff Collection

Scrapes playoff data for multiple seasons from Wikipedia.

In [101]:
# Scrape multiple seasons of playoffs from Wikipedia
import pandas as pd
import time
from datetime import datetime

ensure_environment()

# Define season range (playoffs typically started around 2006-2007)
current_year = datetime.now().year
if datetime.now().month < 8:  # If before August, last season started in previous year
    current_year -= 1

# Start from 2006/07 season when playoffs format was introduced
seasons = list(range(2006, current_year + 1))

print(f"Scraping playoffs for {len(seasons)} seasons ({seasons[0]}/{str(seasons[0]+1)[-2:]} to {seasons[-1]}/{str(seasons[-1]+1)[-2:]})...")
print("="*80)

# Storage for all playoff matches
all_championship = []
all_relegation = []
failed_seasons = []

for season_year in seasons:
    champ_df, releg_df = scrape_season_playoffs(season_year)

    # Save championship round
    if champ_df is not None:
        season_path = DATA_DIR / f"playoffs_championship_{season_year}_{str(season_year+1)[-2:]}_ligat_haal_wikipedia.csv"
        save_csv(champ_df, season_path)
        all_championship.append(champ_df)

    # Save relegation round
    if releg_df is not None:
        season_path = DATA_DIR / f"playoffs_relegation_{season_year}_{str(season_year+1)[-2:]}_ligat_haal_wikipedia.csv"
        save_csv(releg_df, season_path)
        all_relegation.append(releg_df)

    # Track failed seasons
    if champ_df is None and releg_df is None:
        failed_seasons.append(f"{season_year}/{str(season_year+1)[-2:]}")

    time.sleep(1)  # Be nice to Wikipedia

print("\n" + "="*80)

# Combine and save championship rounds
if all_championship:
    combined_champ = pd.concat(all_championship, ignore_index=True)
    combined_path = DATA_DIR / "playoffs_championship_all_seasons_ligat_haal_wikipedia.csv"
    save_csv(combined_champ, combined_path)

    print(f"\n📊 Championship Round Summary:")
    print(f"   Successfully scraped: {len(all_championship)} seasons")
    print(f"   Total matches: {len(combined_champ)}")
    print(f"\n   Matches per season:")
    champ_counts = combined_champ.groupby('season').size().sort_index()
    for season, count in champ_counts.items():
        print(f"      • {season}: {count:3d} matches")
else:
    print("\n❌ No championship playoff matches were successfully scraped")

print("\n" + "-"*80)

# Combine and save relegation rounds
if all_relegation:
    combined_releg = pd.concat(all_relegation, ignore_index=True)
    combined_path = DATA_DIR / "playoffs_relegation_all_seasons_ligat_haal_wikipedia.csv"
    save_csv(combined_releg, combined_path)

    print(f"\n📊 Relegation Round Summary:")
    print(f"   Successfully scraped: {len(all_relegation)} seasons")
    print(f"   Total matches: {len(combined_releg)}")
    print(f"\n   Matches per season:")
    releg_counts = combined_releg.groupby('season').size().sort_index()
    for season, count in releg_counts.items():
        print(f"      • {season}: {count:3d} matches")
else:
    print("\n❌ No relegation playoff matches were successfully scraped")

if failed_seasons:
    print(f"\n⚠️  Seasons with no playoff data: {', '.join(failed_seasons)}")

print("\n" + "="*80)

# Display sample data
if all_championship:
    print(f"\n   Championship Round Sample:")
    display(combined_champ.head(10))

if all_relegation:
    print(f"\n   Relegation Round Sample:")
    display(combined_releg.head(10))

Scraping playoffs for 20 seasons (2006/07 to 2025/26)...
Fetching 2006/07 Championship round... ❌ (no results found)
❌ (no results found)
Fetching 2006/07 Relegation round... Fetching 2006/07 Relegation round... ❌ (no results found)
❌ (no results found)
Fetching 2007/08 Championship round... Fetching 2007/08 Championship round... ❌ (no results found)
❌ (no results found)
Fetching 2007/08 Relegation round... Fetching 2007/08 Relegation round... ❌ (no results found)
❌ (no results found)
Fetching 2008/09 Championship round... Fetching 2008/09 Championship round... ❌ (no results found)
❌ (no results found)
Fetching 2008/09 Relegation round... Fetching 2008/09 Relegation round... ❌ (no results found)
❌ (no results found)
Fetching 2009/10 Championship round... Fetching 2009/10 Championship round... ✓ (15 matches)
✓ (15 matches)
Fetching 2009/10 Relegation round... Fetching 2009/10 Relegation round... ✓ (15 matches)
Saved: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks

,season,season_year,playoff_type,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points
0,2009/10,2009,championship,Beitar Jerusalem,Bnei Yehuda,0,2,-2,A,0,3
1,2009/10,2009,championship,Beitar Jerusalem,Hapoel Tel Aviv,1,2,-1,A,0,3
2,2009/10,2009,championship,Bnei Yehuda,Maccabi Haifa,1,1,0,D,1,1
3,2009/10,2009,championship,Bnei Yehuda,Maccabi Tel Aviv,0,0,0,D,1,1
4,2009/10,2009,championship,F.C. Ashdod,Beitar Jerusalem,1,2,-1,A,0,3
5,2009/10,2009,championship,F.C. Ashdod,Bnei Yehuda,2,3,-1,A,0,3
6,2009/10,2009,championship,Hapoel Tel Aviv,Bnei Yehuda,1,0,1,H,3,0
7,2009/10,2009,championship,Hapoel Tel Aviv,F.C. Ashdod,4,0,4,H,3,0
8,2009/10,2009,championship,Hapoel Tel Aviv,Maccabi Tel Aviv,0,0,0,D,1,1
9,2009/10,2009,championship,Maccabi Haifa,Beitar Jerusalem,2,1,1,H,3,0



   Relegation Round Sample:


,season,season_year,playoff_type,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points
0,2009/10,2009,relegation,Hapoel Acre,Hapoel Haifa,1,0,1,H,3,0
1,2009/10,2009,relegation,Hapoel Acre,Maccabi Ahi Nazareth,3,1,2,H,3,0
2,2009/10,2009,relegation,Hapoel Haifa,Hapoel Ra'anana,1,0,1,H,3,0
3,2009/10,2009,relegation,Hapoel Haifa,Maccabi Ahi Nazareth,2,2,0,D,1,1
4,2009/10,2009,relegation,Hapoel Haifa,Hapoel Petah Tikva,1,2,-1,A,0,3
5,2009/10,2009,relegation,Hapoel Ra'anana,Hapoel Acre,4,0,4,H,3,0
6,2009/10,2009,relegation,Hapoel Ra'anana,Maccabi Ahi Nazareth,2,0,2,H,3,0
7,2009/10,2009,relegation,Hapoel Ramat Gan,Hapoel Acre,0,0,0,D,1,1
8,2009/10,2009,relegation,Hapoel Ramat Gan,Hapoel Haifa,0,1,-1,A,0,3
9,2009/10,2009,relegation,Hapoel Ramat Gan,Hapoel Ra'anana,0,0,0,D,1,1


## 5. Team Name Normalization

th
1
Use the same normalization as regular season to ensure consistency.